In [3]:
from transformers import AutoModel
from transformers import AutoTokenizer


C:\Users\Nick\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
import numpy as np
import torch
import pandas as pd
import pickle

In [112]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam
import time
import pickle

In [75]:
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import ConfusionMatrixDisplay as cmd

In [6]:
from datasets import load_dataset

In [5]:
model = AutoModel.from_pretrained("allenai/scibert_scivocab_cased").cuda()
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_cased")

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
conll = open("nick_merged.conll","rt", encoding="utf-8").read()

In [32]:
text = (conll.split('\n\n'))

In [33]:
X = []
Y = []
labels = []

failed_lines = 0
total_lines = 0
for paragraph in text:
    if len(paragraph) == 0:
        continue
    pg_text = []
    pg_labels = []
    for line in paragraph.split('\n'):
        total_lines += 1
        try:
            pg_text.append(line.split(' ')[0])
            pg_labels.append(line.split(' ')[1])
        except:
            failed_lines += 1
            continue
    X.append(' '.join(pg_text).strip())
    Y.append(pg_labels)
    labels += pg_labels

In [34]:
unique_labels = np.sort(pd.Series(labels).unique())
label_to_ind = {}
ind_to_label = {}
for i in range(len(unique_labels)):
    label_to_ind[unique_labels[i]] = i
    ind_to_label[i] = unique_labels[i]

In [36]:
pickle.dump(ind_to_label, open("ind_to_label.pkl", "wb"))

In [35]:
for i in range(len(Y)):
    for j in range(len(Y[i])):
        Y[i][j] = label_to_ind[Y[i][j]]

In [147]:
token_ids = []
token_labels = []

In [148]:
for i in range(len(X)):
    tokens = tokenizer(X[i].split(' '))['input_ids']
    pg_tokens = [101]
    pg_labels = [label_to_ind['O']]
    for j in range(len(tokens)):
        pg_tokens += tokens[j][1:-1]
        pg_labels += [Y[i][j] for k in range(len(tokens[j]) - 2)]
    pg_tokens += [102]
    pg_labels += [label_to_ind['O']]
    token_ids.append(pg_tokens)
    token_labels.append(pg_labels)

In [153]:
model.requires_grad = False

In [155]:
last_hidden_X = []
labels = []
for i in range(len(token_ids)):
    if i % 100 == 0:
        print(i)
    last_hidden_X.append(model(torch.tensor(token_ids[i][:512], dtype=torch.long).cuda().reshape(1, -1))['last_hidden_state'][0].detach())
    labels += (token_labels[i][:512])

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400


In [156]:
cat_X = torch.cat(last_hidden_X).cuda()
cat_Y = torch.tensor(labels, dtype = torch.long).cuda()

In [53]:
train_ind = int(len(cat_X)*.7)
train_X = cat_X[:train_ind]
val_X = cat_X[train_ind:]
train_Y = cat_Y[:train_ind]
val_Y = cat_Y[train_ind:]

In [55]:
train_data = TensorDataset(train_X, train_Y)
val_data = TensorDataset(val_X, val_Y)
train_loader = DataLoader(train_data, batch_size = 32, shuffle = True)
val_loader = DataLoader(val_data, batch_size = 128, shuffle = True)

In [41]:
fname = "../data/test_data/anlp-sciner-test.txt"

text = open(fname,"rt", encoding="utf-8").read().split('\n')

ind_converter = {}
token_ids = []

for i in range(len(text)):
    ind_converter[i] = {}
    tokens = tokenizer(text[i].split(' '))['input_ids']
    pg_tokens = [101]
    out_ind = 1
    for j in range(len(tokens)):
        ind_converter[i][j] = []
        for k in range(len(tokens[j]) - 2):
            ind_converter[i][j].append(out_ind)
            out_ind += 1
        pg_tokens += tokens[j][1:-1]
    pg_tokens += [102]
    token_ids.append(pg_tokens)

last_hidden_X = []
for i in range(len(token_ids)):
    if i % 100 == 0:
        print(i)
    if len(token_ids[i]) <= 512:
        last_hidden_X.append(model(torch.tensor(token_ids[i][:512], dtype=torch.long).cuda().reshape(1, -1))['last_hidden_state'][0].detach())
    else:
        pred1 = model(torch.tensor(token_ids[i][:512], dtype=torch.long).cuda().reshape(1, -1))['last_hidden_state'][0].detach()
        pred2 = model(torch.tensor(token_ids[i][-512:], dtype=torch.long).cuda().reshape(1, -1))['last_hidden_state'][0].detach()
        pred = torch.cat([pred1, pred2[-(len(token_ids[i]) - 512):]])
        last_hidden_X.append(pred)

0
100
200
300
400
500
600
700
800
900


In [38]:
last_hidden_X[0].shape

torch.Size([17, 768])

In [25]:
test_X = torch.cat(last_hidden_X)

In [26]:
test_X.shape

torch.Size([84018, 768])

In [39]:
pickle.dump(test_X, open("test_X.pkl", "wb"))

In [42]:
pickle.dump(last_hidden_X, open("last_hidden_X.pkl", "wb"))